In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
train  = pd.read_excel(r'D:\projects\aviation\dataset\PM_train.xlsx')
truth = pd.read_excel(r'D:\projects\aviation\dataset\PM_truth.xlsx')

train.head(5)
truth.head(5)

print(train.shape)
print(truth.shape)


(20631, 26)
(99, 2)


In [29]:
dup_rows = train.duplicated(subset=['id','cycle']).sum()
print(dup_rows)


missing = train.isna().sum()
print(missing)


def check_cycle(df):
    return df.groupby('id')['cycle'].apply(lambda x:x.is_monotonic_increasing)

cycle_check = check_cycle(train)
print("Engine with cycle issues",cycle_check.sum())

0
id          0
cycle       0
setting1    0
setting2    0
setting3    0
s1          0
s2          0
s3          0
s4          0
s5          0
s6          0
s7          0
s8          0
s9          0
s10         0
s11         0
s12         0
s13         0
s14         0
s15         0
s16         0
s17         0
s18         0
s19         0
s20         0
s21         0
dtype: int64
Engine with cycle issues 100


In [33]:
train_sorted = train.sort_values(["id", "cycle"])
cycle_check_fixed = train_sorted.groupby("id")["cycle"] \
    .apply(lambda x: x.is_monotonic_increasing)

print("Engines with cycle issues AFTER sorting:", (~cycle_check_fixed).sum())


Engines with cycle issues AFTER sorting: 0


In [59]:
sensor_cols =[c for c in train.columns if c.startswith('s')]

sensor_variance =train[sensor_cols].var()
low_var_sens = sensor_variance[sensor_variance < 1e-4]


low_var_sens






setting1    4.784340e-06
setting2    8.588541e-08
setting3    0.000000e+00
s1          4.273435e-21
s5          1.152399e-23
s6          1.929279e-06
s10         2.172333e-25
s16         2.422479e-28
s18         0.000000e+00
s19         0.000000e+00
dtype: float64

In [73]:
max_cycles = train.groupby('id')["cycle"].max().reset_index()
max_cycles.columns = ['id','max_cycle']





,id,max_cycle
0,1,192
1,2,287
2,3,179
3,4,189
4,5,269
...,...,...
95,96,336
96,97,202
97,98,156
98,99,185


In [75]:
train.drop(columns=["max_cycle"], inplace=True)
train.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,max_cycle_x,RUL,max_cycle_y
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100,39.06,23.4190,192,191,192
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100,39.00,23.4236,192,190,192
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100,38.95,23.3442,192,189,192
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100,38.88,23.3739,192,188,192
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100,38.90,23.4044,192,187,192


In [83]:
train.to_csv(r"D:\projects\aviation\dataset/clean_train_data.csv", index=False)

engine_summary = train.groupby("id").agg(
    max_cycle=("cycle", "max"),
    min_RUL=("RUL", "min")
).reset_index()

engine_summary.to_csv(r"D:\projects\aviation\dataset/engine_lifecycle_summary.csv", index=False)


In [85]:
pd.read_csv(r"D:\projects\aviation\dataset/engine_lifecycle_summary.csv")

,id,max_cycle,min_RUL
0,1,192,0
1,2,287,0
2,3,179,0
3,4,189,0
4,5,269,0
...,...,...,...
95,96,336,0
96,97,202,0
97,98,156,0
98,99,185,0
